- normalizing the images
- increasing the epochs 

In [1]:
from work_2.helper import *
import tensorflow as tf 
import numpy as np 
from work_2.model import *
import pandas as pd

In [2]:
imglist = img_location_reader("data")
train,test = train_test_split(imglist,test_size=0.3,random_state=0)
train,valid = train_test_split(train,test_size=0.3,random_state=0)

In [3]:
sum([len(train[i]) for i in range(len(train))]) # total images in train

2069

In [4]:
X = tf.placeholder(tf.float32,[None,256,256,3],name = "Input_data")
Y = tf.placeholder(tf.float32,[None,8], name = "InputLabels")

In [5]:
# defining weights and bias 
weights = {
"w1": init_weights([3,3,3,16]),
"w2": init_weights([2,2,16,16]),
"w3": init_weights([2,2,16,32]),
"w4": init_weights([2,2,32,32]),
"w5": init_weights([2,2,32,64]),
"w6": init_weights([2,2,64,64]),
"w7": init_weights([256,256]),
"w8": init_weights([256,64]),
"w9": init_weights([64,8])
}

bias = {
"b1":init_bias([16]),
"b2":init_bias([16]),
"b3":init_bias([32]),
"b4":init_bias([32]),
"b5":init_bias([64]),
"b6":init_bias([64]),
"b7":init_bias([256]),
"b8":init_bias([64]),
"b9":init_bias([8])
}

In [6]:
pred = model(X,weights["w1"],weights["w2"],weights["w3"],weights["w4"],weights["w5"],weights["w6"],
weights["w7"],weights["w8"],weights["w9"],bias["b1"],bias["b2"],bias["b3"],bias["b4"],bias["b5"],bias["b6"],bias["b7"],
bias["b8"],bias["b9"])

In [7]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [8]:
correct_pred= tf.equal(tf.argmax(pred,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [9]:
#Initialize all the variables
init = tf.initialize_all_variables()

saver = tf.train.Saver()

# Create summary to monitor cost tensor , accuracy tensor
tf.scalar_summary("loss",cost)
tf.scalar_summary("accuracy",accuracy)

merged_summary_op = tf.merge_all_summaries()

In [10]:
# splitting is done .
img_list = img_location_list(train,over_sample = True)

In [11]:
img_dummy_label = pd.get_dummies(list(image_label(img_list).keys()))

In [12]:
img_dummy_label.columns

Index(['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'], dtype='object')

In [13]:
x_val,y_val = val_test_image_reader(valid)
x_test,y_test = val_test_image_reader(test)
x_val = x_val / 255
x_test = x_test / 255

In [14]:
print ("x_val:",x_val.shape,"y_val: ",y_val.shape)
print ("x_test:",x_test.shape,"y_test: ",y_test.shape)

x_val: (832, 256, 256, 3) y_val:  (832, 8)
x_test: (1130, 256, 256, 3) y_test:  (1130, 8)


In [15]:
epochs = 20
batch_size = 8
logs_path = "/tmp/tensorflow_logs/example"
model_path = "/models/run2"

In [16]:
with tf.Session() as sess:
    sess.run(init)
    summary_writer = tf.train.SummaryWriter(logs_path,graph=tf.get_default_graph())
    for ep in tqdm(range(epochs)):
        img_list = shuffle(img_list,random_state=0)
        for i in range(int(len(img_list)/batch_size)):
            img_loc = img_list[i*batch_size:(i+1)*batch_size]
            X_image, Y_image = dev_image_reader(img_loc,img_dummy_label)
            X_image, Y_image = X_image/255, Y_image
            _,c,summary = sess.run([optimizer,cost,merged_summary_op],feed_dict={X : np.float32(X_image), Y : np.float32(Y_image)})
            train_loss, train_acc = sess.run([cost,accuracy], feed_dict={X : np.float32(X_image), Y : np.float32(Y_image)})
            if i % 100 == 0:
                print ("Train loss ="+"{:.6f}".format(train_loss),\
               "Train acc ="+"{:.6f}".format(train_acc))
        print ("epoch: ", ep)
        valid_loss, valid_acc = sess.run([cost,accuracy], feed_dict={X : np.float32(x_val), Y : np.float32(y_val)})
        print ("Train loss ="+"{:.6f}".format(train_loss),\
            "Train acc ="+"{:.6f}".format(train_acc),\
            "valid loss ="+"{:.6f}".format(valid_loss),\
            "valid acc ="+"{:.6f}".format(valid_acc))
    print("Optimization Finished!")
    test_loss, test_acc = sess.run([cost,accuracy], feed_dict={X : np.float32(x_test), Y : np.float32(y_test)})
    print ("test loss ="+"{:.6f}".format(test_loss),\
        "Test acc ="+"{:.6f}".format(test_acc))
    save_path = saver.save(sess,os.getcwd()+model_path)


  0%|          | 0/20 [00:00<?, ?it/s]

Train loss =2.078648 Train acc =0.187500
Train loss =2.078940 Train acc =0.250000
Train loss =2.081948 Train acc =0.125000
Train loss =2.083656 Train acc =0.000000
Train loss =2.080413 Train acc =0.125000
Train loss =2.076529 Train acc =0.250000
Train loss =2.079875 Train acc =0.000000
Train loss =2.077776 Train acc =0.250000
Train loss =2.079024 Train acc =0.125000
Train loss =2.079611 Train acc =0.000000
epoch:  0


  5%|▌         | 1/20 [03:10<1:00:15, 190.29s/it]

Train loss =2.081024 Train acc =0.125000 valid loss =2.078832 valid acc =0.031250
Train loss =2.078156 Train acc =0.125000
Train loss =2.082022 Train acc =0.000000
Train loss =2.079781 Train acc =0.000000
Train loss =2.082625 Train acc =0.125000
Train loss =2.084568 Train acc =0.000000
Train loss =2.072439 Train acc =0.375000
Train loss =2.075184 Train acc =0.125000
Train loss =2.077550 Train acc =0.125000
Train loss =2.079360 Train acc =0.000000
Train loss =2.079107 Train acc =0.125000
epoch:  1


 10%|█         | 2/20 [06:19<57:00, 190.04s/it]  

...

 95%|█████████▌| 19/20 [59:44<03:08, 188.38s/it]

Train loss =2.080837 Train acc =0.000000 valid loss =2.075507 valid acc =0.457933
Train loss =2.077288 Train acc =0.125000
Train loss =2.073731 Train acc =0.375000
Train loss =2.083768 Train acc =0.125000
Train loss =2.080063 Train acc =0.125000
Train loss =2.071187 Train acc =0.125000
Train loss =2.075912 Train acc =0.250000
Train loss =2.079623 Train acc =0.000000
Train loss =2.081496 Train acc =0.250000
Train loss =2.077205 Train acc =0.125000
Train loss =2.077865 Train acc =0.125000
epoch:  19


100%|██████████| 20/20 [1:02:53<00:00, 188.36s/it]

Train loss =2.080482 Train acc =0.000000 valid loss =2.079019 valid acc =0.016827
Optimization Finished!


test loss =2.079025 Test acc =0.017699
